# Part 1 Data Ingestion & Storage

Import Libraries

In [ ]:
#pip install streamlit plotly nbformat

In [ ]:
#pip install duckdb pandas requests

In [ ]:
import os
import requests
import pandas as pd
import duckdb
from pathlib import Path
import nbformat

### Create Data Directory

In [ ]:
raw_path = Path("data/raw")
raw_path.mkdir(parents=True, exist_ok=True)

### Programmatic Download

Approach and rationale: The dataset is downloaded programmatically to ensure the pipeline is reproducible. Saving the raw files into data/raw/ separates immutable source data from later processed outputs, which makes it easier to debug issues during cleaning and ensures downstream steps can always be re-run from a known starting point.

Download the dataset if it doesn't exist

In [ ]:
def download_file(url, save_path):
    if not save_path.exists():
        print(f"Downloading {save_path.name}...")
        response = requests.get(url)
        response.raise_for_status()
        with open(save_path, "wb") as f:
            f.write(response.content)
        print("Download complete.")
    else:
        print(f"{save_path.name} already exists.")

Define URLS and download

In [ ]:
trip_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
zone_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

download_file(trip_url, raw_path / "yellow_tripdata_2024-01.parquet")
download_file(zone_url, raw_path / "taxi_zone_lookup.csv")

### Load Data

In [ ]:
df = pd.read_parquet(raw_path / "yellow_tripdata_2024-01.parquet")
zones = pd.read_csv(raw_path / "taxi_zone_lookup.csv")

### Data Validation

Validation checks: Before transforming the dataset, I verify that all required schema columns are present and that pickup and dropoff timestamps parse correctly as datetimes. These checks prevent silent errors later (for example, if a column name changes or a timestamp fails to parse). If validation fails, the notebook raises an exception so that incorrect data does not propagate into cleaning, analysis, or the dashboard.

Checks required columns from Schema

In [ ]:
required_columns = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "passenger_count",
    "trip_distance",
    "fare_amount",
    "tip_amount",
    "total_amount",
    "payment_type"
]

missing_columns = set(required_columns) - set(df.columns)

if missing_columns:
    raise ValueError(f"Missing columns: {missing_columns}")
else:
    print("All required columns are present.")


Standardizing datetime into datetime objects

In [ ]:
df["tpep_pickup_datetime"] = pd.to_datetime(
    df["tpep_pickup_datetime"], errors="raise"
)

df["tpep_dropoff_datetime"] = pd.to_datetime(
    df["tpep_dropoff_datetime"], errors="raise"
)

Print Summary of Data

In [ ]:
print(f"Total rows: {len(df):,}")
df.describe()

# Part 2: Data Transformation & Analysis

### Data Cleaning 

Cleaning decisions and observations: Taxi trip data contains invalid or unusable records such as missing critical fields, non-positive distances, and fares outside expected bounds. I remove rows with null values in critical columns, filter out trips with zero/negative distance or invalid fare values, cap fares above $500 as extreme outliers, and remove trips where dropoff occurs before pickup. After cleaning, I print counts for each removal category to document how the dataset changed and to justify the cleaning rules.

Counting values to be removed 

In [ ]:
initial_rows = len(df)

null_rows = df[
    df[[
        "tpep_pickup_datetime",
        "tpep_dropoff_datetime",
        "PULocationID",
        "DOLocationID",
        "fare_amount"
    ]].isnull().any(axis=1)
]

negative_distance = df[df["trip_distance"] <= 0]
negative_fare = df[df["fare_amount"] <= 0]
extreme_fare = df[df["fare_amount"] > 500]
invalid_time = df[df["tpep_dropoff_datetime"] <= df["tpep_pickup_datetime"]]

Remove Null Values

In [ ]:
initial_rows = len(df)

df = df.dropna(subset=[
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "fare_amount"
])

Remove invalid trips

In [ ]:
df = df[
    (df["trip_distance"] > 0) &
    (df["fare_amount"] > 0) &
    (df["fare_amount"] <= 500) &
    (df["tpep_dropoff_datetime"] > df["tpep_pickup_datetime"])
]

Cleaning Summary: Shows rows removed and why

In [ ]:
final_rows = len(df)

print(f"Total Rows removed: {initial_rows - final_rows}")
print("Rows with nulls:", len(null_rows))
print("Rows with non-positive distance:", len(negative_distance))
print("Rows with non-positive fare:", len(negative_fare))
print("Rows with extreme fare (>500):", len(extreme_fare))
print("Rows with invalid timestamps:", len(invalid_time))
print(f"Remaining rows: {final_rows:,}")

### Feature Engineering

Feature engineering rationale: The derived columns are designed to support the specific dashboard analyses required. Trip duration and speed summarize trip dynamics and support metrics and distribution checks. Pickup hour and pickup day-of-week enable temporal aggregation for hourly and weekly patterns. Division by zero is handled by converting zero-duration trips into missing values so that speed does not become infinite and distort summary statistics.

#### 4 Features:

In [ ]:
df["trip_duration_minutes"] = (
    df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
).dt.total_seconds() / 60

df["trip_speed_mph"] = df["trip_distance"] / (
    df["trip_duration_minutes"].replace(0, pd.NA) / 60
)

df["pickup_hour"] = df["tpep_pickup_datetime"].dt.hour

df["pickup_day_of_week"] = df["tpep_pickup_datetime"].dt.day_name()

Note:
Zero-duration trips would cause division by zero when calculating speed. These durations were replaced with pd.NA so that Pandas safely propagates missing values instead of producing infinite speeds.

Verify features

In [ ]:
df[[
    "trip_duration_minutes",
    "trip_speed_mph",
    "pickup_hour",
    "pickup_day_of_week"
]].head()


This shows the top few entries per category, hence proving that the features have been successfully created.

### SQL Analysis

Interpreting the SQL results: The DuckDB queries answer targeted analytical questions about demand concentration (busiest zones), fare dynamics (average fare by hour), rider behavior (payment mix), tipping patterns (credit-card tip percentage by weekday), and common travel flows (pickup-dropoff pairs). These results directly inform which visualizations are most useful to include in the dashboard and what insights to highlight beneath them.

Create a duckdb connection

In [ ]:
con = duckdb.connect()
con.register("taxi", df)
con.register("zones", zones)

### Query 1
What are the top 10 busiest pickup zones by total number of trips? 

In [ ]:
query1 = """
SELECT z.Zone, COUNT(*) AS trips
FROM taxi t
JOIN zones z
ON t.PULocationID = z.LocationID
GROUP BY z.Zone
ORDER BY trips DESC
LIMIT 10;
"""

top_zones = con.execute(query1).df()
top_zones


The top pickup zones are Midtown Center (140,141 trips), Upper East Side South (140,118), and JFK Airport (138,427), followed closely by Upper East Side North and Midtown East. Several Manhattan commercial districts and both major airports (JFK and LaGuardia) appear in the top ten.

This shows that taxi demand is highly concentrated in central Manhattan and major transportation hubs. Midtown Center and Upper East Side South are nearly tied for the highest trip counts, indicating strong activity in dense residential and commercial areas. The presence of JFK Airport and LaGuardia Airport among the top pickup zones highlights the importance of airport-related travel in overall taxi demand.

Notably, multiple Midtown and Upper East/West Side zones appear in the top ten, suggesting that Manhattan dominates pickup activity. These areas likely reflect a combination of commuting, tourism, and business travel. The inclusion of Penn Station/Madison Square West and Times Square/Theatre District further supports the idea that transit hubs and entertainment districts generate significant ride volume.

The relatively small difference between the top four zones (all above 133,000 trips) suggests that demand is intense but somewhat evenly distributed among key Manhattan neighborhoods. However, there is a noticeable drop after the top four, indicating that demand becomes more dispersed outside the most central zones.

This result justifies including geographic filters in the dashboard, since user-selected pickup zones can reveal how demand patterns differ between airport travel, commuter-heavy areas, and residential neighborhoods.

### Query 2
What is the average fare amount for each hour of the day? 

In [ ]:
query2 = """
SELECT pickup_hour,
       ROUND(AVG(fare_amount),2) AS avg_fare
FROM taxi
GROUP BY pickup_hour
ORDER BY pickup_hour;
"""

avg_fare_hour = con.execute(query2).df()
avg_fare_hour

The lowest average fare occurs around 2 AM ($16.63), while the highest average fare occurs at 5 AM ($27.50). There is also a noticeable spike at 4 AM ($23.45). During most daytime hours (7 AM–8 PM), the average fare remains relatively stable between approximately $17 and $20.

The hourly pattern shows a clear early-morning spike in average fare between 4 AM and 6 AM, peaking at 5 AM. This is interesting because it does not align with traditional commuting hours. A likely explanation is that early-morning trips are longer-distance rides, possibly airport-bound trips or long cross-borough travel when traffic is minimal. Lower traffic at that time may encourage longer trips, which increases the average fare.

Between approximately 7 AM and 8 PM, the average fare stabilizes in a narrow band around $17–$19. This suggests that daytime trips are more routine, shorter urban trips within Manhattan and nearby areas. The relative stability during business hours indicates consistent trip patterns driven by work, errands, and tourism.

The slight increase again late at night (around 11 PM, $20.25) may reflect nightlife-related longer rides or late airport departures.

Overall, the data suggests that trip length - rather than traffic congestion alone - plays a major role in determining hourly fare averages. The early-morning spike is particularly important because it reveals a non-obvious pattern that would not be immediately visible without aggregation.

This supports including the hourly trend line in the dashboard, as filtering by pickup zone or payment type may further clarify whether the early-morning increase is driven primarily by airport zones or specific neighborhoods.

### Query 3
What percentage of trips use each payment type?

In [ ]:
query3 = """
SELECT payment_type,
       COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS percentage
FROM taxi
GROUP BY payment_type
ORDER BY percentage DESC;
"""

payment_pct = con.execute(query3).df()
payment_pct

Payment type 1 accounts for approximately 80.09% of trips, payment type 2 accounts for 14.73%, and the remaining categories (0, 4, and 3) each account for less than 5%, with two of them below 1%.

The results show a strong dominance of credit card payments, which account for approximately 80% of all trips. This indicates that electronic payments are the primary transaction method in NYC taxi operations. Cash payments make up roughly 15% of trips, representing a significant but much smaller share of overall transactions.

The remaining payment categories - including unknown, disputed, and no-charge trips - collectively account for less than 6% of total rides. Their small percentages suggest that most trips are processed through standard, completed transactions.

The overwhelming dominance of credit card payments has important implications for downstream analysis. For example, tip data is more reliably recorded for credit card transactions, which explains why tipping behavior analysis is restricted to payment type 1. It also suggests that digital transaction systems play a major role in NYC taxi operations.

From a dashboard perspective, including payment-type filters allows users to compare behavioral patterns between credit card and cash users. Given the large difference in volume between these two categories, filtering may reveal meaningful differences in trip distance, fare averages, or tipping patterns.

### Query 4
What is the average tip percentage (tip_amount/fare_amount) by day of week, for credit card payments only?

In [ ]:
query4 = """
SELECT pickup_day_of_week,
       ROUND(AVG(tip_amount/fare_amount)*100,2) AS avg_tip_pct
FROM taxi
WHERE payment_type = 1
GROUP BY pickup_day_of_week;
"""

tip_by_day = con.execute(query4).df()
tip_by_day

Average tip percentages range from approximately 25.10% (Sunday) to 29.73% (Thursday). Most days cluster tightly between 25% and 26%, with Thursday standing out as noticeably higher.

The results show that average tip percentages are relatively stable across most days of the week, generally falling between 25% and 26%. This suggests that tipping behavior among credit card users is fairly consistent regardless of weekday. However, Thursday stands out with a noticeably higher average tip percentage of 29.73%, which is approximately 3–4 percentage points above the other days.

This elevated Thursday tipping rate may reflect behavioral or contextual factors. Thursdays often mark the beginning of increased social activity (e.g., dining, entertainment, early weekend travel), which may influence tipping generosity. Alternatively, it could be associated with specific trip types such as airport travel or business-related rides.

The fact that Sunday has the lowest average tip percentage (25.10%) suggests that tipping may slightly decrease at the end of the weekend. However, the overall variation across days is relatively modest, indicating that tipping behavior is generally stable throughout the week.

Because this analysis is restricted to credit card payments, the tip percentage values are reliable and not affected by unrecorded cash tips. Including weekday patterns in the dashboard allows users to visually confirm whether higher-demand days correspond to increased tipping behavior.

### Query 5
What are the top 5 most common pickup-dropoff zone pairs?

In [ ]:
query5 = """
SELECT z1.Zone AS pickup_zone,
       z2.Zone AS dropoff_zone,
       COUNT(*) AS trips
FROM taxi t
JOIN zones z1 ON t.PULocationID = z1.LocationID
JOIN zones z2 ON t.DOLocationID = z2.LocationID
GROUP BY pickup_zone, dropoff_zone
ORDER BY trips DESC
LIMIT 5;
"""

top_routes = con.execute(query5).df()
top_routes


The most frequent route is Upper East Side South to Upper East Side North (21,641 trips), followed by the reverse direction (19,199 trips). Several of the top routes are intra–Upper East Side trips or travel between neighboring Upper East Side zones. Midtown Center to Upper East Side South also appears in the top five.

The results show that the most common taxi trips occur within or between closely adjacent Upper East Side zones. The two highest routes are reciprocal flows between Upper East Side South and Upper East Side North, indicating heavy local movement within that neighborhood. This suggests that taxi usage in this area is driven by short-distance travel, possibly for convenience, shopping, dining, or residential mobility.

Notably, intra-zone trips (North → North and South → South) also appear in the top five. This indicates that many taxi rides occur entirely within the same neighborhood zone, reinforcing the idea that taxis are frequently used for short urban trips rather than exclusively long-distance travel.

The appearance of Midtown Center → Upper East Side South in the top five highlights a strong connection between commercial/business districts and residential neighborhoods. This likely reflects commuting or business-related travel patterns.

Overall, these findings suggest that taxi travel is highly localized and structured around predictable urban corridors rather than random city-wide movement. The dominance of Upper East Side flows indicates strong neighborhood-based demand patterns. In the dashboard context, zone filters allow users to explore whether these route patterns remain dominant across different hours of the day or payment types.

The taxi zone lookup table was joined to the trip dataset to add human-readable pickup zone names required for dashboard visualizations. A separate pickup_date column was extracted to enable date-based filtering in Streamlit.

In [ ]:
df = df.merge(
    zones[['LocationID', 'Zone']],
    left_on='PULocationID',
    right_on='LocationID',
    how='left'
)

df = df.rename(columns={'Zone': 'pickup_zone'})

df['pickup_date'] = df['tpep_pickup_datetime'].dt.date

Saves processed dataset data/processed/cleaned_taxi_2024_01.parquet

In [ ]:
processed_path = Path("data/processed")
processed_path.mkdir(parents=True, exist_ok=True)

df.to_parquet(processed_path / "cleaned_taxi_2024_01.parquet", index=False)

print("Cleaned dataset saved successfully.")

Query showing distribution in trips by year and month.

In [ ]:
query_monthly = """
SELECT 
    EXTRACT(YEAR FROM tpep_pickup_datetime) AS year,
    EXTRACT(MONTH FROM tpep_pickup_datetime) AS month,
    COUNT(*) AS total_trips
FROM taxi
GROUP BY year, month
ORDER BY year, month;
"""

monthly_trips = con.execute(query_monthly).df()
monthly_trips

This Query shows the distribution of data amongst years and months, it was trivial in the creation if the date range filter to understannd where the data was as its scarcely distributed among the years beside 2024 january

# Part 3 Dashboard Developmemnt

In this section, I prototype the visualizations required for the Streamlit dashboard using the cleaned dataset. The purpose of prototyping here is to confirm the transformations and aggregations are correct, verify that each chart communicates the intended pattern, and decide on appropriate formatting (e.g., bin sizes for histograms and category ordering for weekdays) before implementing the final dashboard in app.py. All visualizations shown below are later implemented in Streamlit with interactive filters so the figures update based on user selections.

Import plotly (prototype)

In [ ]:
import plotly.express as px

### Prototype Visualization 1: Top 10 Pickup Zones

This chart shows which pickup zones have the highest trip volume. Identifying the busiest pickup areas gives a quick view of where demand concentrates geographically in NYC.

In [ ]:
top_zones_viz = (
    df["pickup_zone"]
    .value_counts()
    .head(10)
    .reset_index()
)
top_zones_viz.columns = ["Pickup Zone", "Trips"]

fig = px.bar(top_zones_viz, x="Pickup Zone", y="Trips", title="Top 10 Pickup Zones by Trip Count")
fig.show()

The top four pickup zones, Midtown Center, Upper East Side South, JFK Airport, and Upper East Side North each record more than 130,000 trips, indicating extremely high and relatively similar levels of demand across these areas. Midtown Center and Upper East Side South are nearly tied for the highest volume, suggesting that both commercial business districts and dense residential neighborhoods contribute heavily to taxi activity. The inclusion of both JFK and LaGuardia Airports within the top ten highlights the significant role of airport travel in overall trip volume. Additionally, several Midtown and Upper East/West Side zones appear in the ranking, confirming that Manhattan dominates pickup activity. The noticeable drop after the top four zones indicates that demand becomes more dispersed outside the most central and transit-connected areas. Filtering the dashboard by hour may reveal whether airport zones dominate during early morning and late-night hours, while residential zones may peak during commuting periods.

### Prototype Visualization 2: Average Fare by Hour of Day

This line chart summarizes how average fare changes across the day. It helps reveal hourly patterns that may be driven by commuting, traffic, or longer trips at specific times.

In [ ]:
avg_fare_hour_viz = (
    df.groupby("pickup_hour")["fare_amount"]
    .mean()
    .reset_index()
)

fig = px.line(avg_fare_hour_viz, x="pickup_hour", y="fare_amount", markers=True, title="Average Fare by Pickup Hour")
fig.update_layout(xaxis_title="Pickup Hour", yaxis_title="Average Fare ($)")
fig.show()

Average fare varies noticeably throughout the day, with a clear peak at 5 AM where the average fare reaches approximately $27.50, significantly higher than the typical daytime range of $17-$20. There is also a secondary elevation around 4 AM and 6 AM, suggesting that early-morning trips tend to be longer or higher-value rides. After 7 AM, fares stabilize and remain relatively consistent throughout standard daytime hours, indicating more routine urban travel patterns. The slight increase again near 11 PM may reflect late-night travel or airport-bound rides. The pronounced early-morning spike suggests that trip distance, rather than congestion, may be the primary driver of higher fares during that period. In the dashboard, filtering by hour can help determine whether this spike is concentrated in airport pickup zones or specific neighborhoods.

### Prototype Visualization 3: Distribution of Trip Distances

Taxi trip distance is typically right-skewed: many short trips and fewer long trips. To make the histogram readable, extreme outliers can stretch the x-axis; therefore, I cap the display at a high percentile (99th) so the main distribution is visible while still using a true histogram of binned counts.

In [ ]:
max_distance = df["trip_distance"].quantile(0.99)
dist_df = df[df["trip_distance"] <= max_distance]

fig = px.histogram(dist_df, x="trip_distance", nbins=40, title="Trip Distance Distribution (Trimmed at 99th Percentile)")
fig.update_layout(xaxis_title="Trip Distance (miles)", yaxis_title="Number of Trips")
fig.show()

The distribution is heavily right-skewed, with the vast majority of trips concentrated under approximately 3 miles. The highest frequency bins occur in the 1–2 mile range, indicating that short intra-neighborhood rides dominate overall taxi usage. After about 4–5 miles, trip counts decline sharply, and frequencies continue to taper off gradually as distance increases. Even after trimming at the 99th percentile (around 20 miles), a visible right tail remains, suggesting a consistent but much smaller share of longer-distance trips. These longer rides likely correspond to airport travel or cross-borough routes. Trimming at the 99th percentile prevents extreme outliers from compressing the main distribution, allowing the core urban travel pattern to be clearly observed while still preserving the underlying histogram structure.

### Prototype Visualization 4: Payment Type Breakdown

This chart shows how riders pay for trips. It is useful for understanding whether digital payments dominate and whether certain payment categories are rare.

In [ ]:
payment_counts_viz = df["payment_type"].value_counts().reset_index()
payment_counts_viz.columns = ["payment_type", "Trips"]

fig = px.bar(payment_counts_viz, x="payment_type", y="Trips", title="Payment Type Breakdown")
fig.update_layout(xaxis_title="Payment Type Code", yaxis_title="Number of Trips")
fig.show()

The distribution of payment types is highly concentrated in code 1 (Credit Card), which overwhelmingly dominates the dataset with well over 2 million trips. In comparison, cash payments (code 2) account for a substantially smaller but still significant share, while codes 0, 3, and 4 represent only a very small fraction of total trips. The stark difference in bar heights clearly shows that electronic payments are the primary transaction method in NYC taxi operations.

The relatively small presence of dispute, no-charge, and unknown payment categories suggests that the vast majority of trips are completed through standard payment channels. The dominance of credit card payments also explains why tip percentage analysis is more reliable when restricted to payment type 1, since digital payments automatically record tip amounts.

In the Streamlit dashboard, converting numeric payment codes into readable labels (e.g., Credit Card, Cash) improves clarity and makes it easier for users to compare behavioral patterns across payment methods.

### Prototype Visualization 5: Trips by Day of Week and Hour

This heatmap shows the interaction between day-of-week and hour-of-day, highlighting weekly patterns (e.g., weekday commuting peaks versus weekend late-night activity).

In [ ]:
heatmap_data = (
    df.groupby(["pickup_day_of_week", "pickup_hour"])
    .size()
    .reset_index(name="Trips")
)

fig = px.density_heatmap(
    heatmap_data,
    x="pickup_hour",
    y="pickup_day_of_week",
    z="Trips",
    title="Trips by Day of Week and Hour",
    labels={"pickup_hour": "Pickup Hour", "pickup_day_of_week": "Day of Week", "Trips": "Number of Trips"}
)
fig.show()

The heatmap shows that trip volume increases significantly during midday and early evening hours, particularly between approximately 10 AM and 4 PM. The brightest regions (highest trip counts) appear consistently across weekdays during these hours, indicating that daytime demand is stronger than late-night demand. In contrast, the darkest cells occur during very early morning hours (around 12 AM–5 AM), especially midweek, confirming that overnight demand is substantially lower.

Weekday patterns appear relatively consistent, with strong midday peaks across Monday through Friday. Saturday also shows elevated activity during afternoon hours, while Sunday displays slightly lower intensity overall compared to peak weekday levels. The absence of extremely sharp morning spikes suggests that taxi usage may be distributed more evenly throughout the day rather than dominated by narrow commuter rush windows.

Overall, the heatmap highlights that NYC taxi demand is highest during daytime commercial and activity hours rather than strictly early-morning commuting periods. In the dashboard, applying filters by payment type or pickup zone could reveal whether airport zones contribute disproportionately to specific high-intensity periods.

### Implementation Note

After validating these visualizations in the notebook, they were implemented in app.py using Streamlit tabs and interactive filters (date range, hour range, and payment type). Streamlit’s caching is used to avoid repeatedly reloading the dataset, and all charts update based on the filtered dataset.

### AI Tools Used:
ChatGPT was used to assist with:
SQL query refinement  
Debugging environment and dependency issues  
Improving visualization design  
Refining analytical insight statements  
Structuring documentation (README and notebook explanations)  
All analysis logic and implementation decisions were reviewed and validated.  